In [2]:
import redis
import json

In [3]:
r = redis.Redis(host='redis-12442.c90.us-east-1-3.ec2.cloud.redislabs.com',port=12442,password='RHFgwK2pQ2ibFUqw8zrQBHRzK29lQtDw')
keys=r.keys('*') #getting the keys of each convesation
print(keys)

[b'tracker:51862fcd04ac4f0491f701efbdf6c9e0', b'tracker:bd09487cb6314e52b2bda5892e8fbb3f', b'tracker:ffc9da9f36c04e1fb618a444d8143982', b'tracker:a8dcbf90b47f4767a554e41f8e5483d7', b'tracker:e047accea70f4b9ea2a7cf70bbcf8214', b'tracker:1b394dd25cba49c289e1f81458a3f260', b'tracker:6a89a09873af4d5797ada863a91ffd81', b'tracker:3ecb302b489f4eb1a90fdf5bda32cc55', b'tracker:fc5c9514313741c2a8f6894e4e66e405', b'tracker:fe1b187fde2040afb1211c3195010a96', b'tracker:a6b4b897057143ffa56c5c6e11d731d4', b'tracker:f23cbe59dc4947c8a8dc66781fa25c4d', b'tracker:508afa435ed0400eaae21c5d8f68d74c', b'tracker:3ba31dffd42245deb7b83c6bd6a68990', b'tracker:b1c3a26a4f17431f8579fa0156738456', b'tracker:a84421c79fae47348b09a2385c80aadd', b'tracker:9523f1d1624246cba19e075ecaa2b973', b'tracker:ef89910c5c244074918c5db4d8837782', b'tracker:9b197abaf6b04a2e8cd33ef7c149d052', b'tracker:49c872c0e39c4efca64d78faf40f0475', b'tracker:de75cadbdc4e438ea29343515a2c42e4', b'tracker:8452587c6b5140a08bbaa6806b4856e6', b'tracker

In [4]:
trackerdata=[]
for i in keys: 
    temp=r[i]
    temp = temp.decode("utf-8")
    trackerdata.append(json.loads(temp)) #each conversation is saved with its events as a list and trackername
conversations=[]
for i in trackerdata:
    conversations.append(i["events"]) #extracting only the events list of each conversation

In [5]:
for i in conversations[0]:
    print(i)

{'event': 'action', 'timestamp': 1686973941.9463155, 'metadata': {'model_id': '1372581223704efea7014a0251aff89e', 'assistant_id': '20230421-154826-finite-valuation'}, 'name': 'action_session_start', 'policy': None, 'confidence': 1.0, 'action_text': None, 'hide_rule_turn': False}
{'event': 'session_started', 'timestamp': 1686973941.9463155, 'metadata': {'model_id': '1372581223704efea7014a0251aff89e', 'assistant_id': '20230421-154826-finite-valuation'}}
{'event': 'action', 'timestamp': 1686973941.9463155, 'metadata': {'model_id': '1372581223704efea7014a0251aff89e', 'assistant_id': '20230421-154826-finite-valuation'}, 'name': 'action_listen', 'policy': None, 'confidence': None, 'action_text': None, 'hide_rule_turn': False}
{'event': 'user', 'timestamp': 1686973942.4549403, 'metadata': {'model_id': '1372581223704efea7014a0251aff89e', 'assistant_id': '20230421-154826-finite-valuation'}, 'text': 'hi', 'parse_data': {'intent': {'name': 'greet', 'confidence': 0.9998519420623779}, 'entities': [

In [6]:
for conv in conversations:
    for i in conv:
        if i['event']=='user':
            print("user: ",i['text'])
            if len(i['parse_data']['entities'])>0:
                print('data entered: ',i['parse_data']['entities'][0]['entity'],"=",i['parse_data']['entities'][0]['value'])
        elif i['event']=="bot":
            print("bot: ",i["text"])
    print("\n")

user:  hi
bot:  Hey! How are you?
user:  item info
bot:  which of the details of the below you want :
user:  /price_details
bot:  Select Get item prices, to retrieve prices for all items at all locations.

Select Get specific price of item/loc, to retrieve the price of a specific item at a specific location.

Select Get price of item at all locs, to retrieve prices for a specific item at all available locations.

Please select a use case:
user:  /get_item_prices
bot:  use case 1
bot:  item_no: 100190308
location: 102
location_type: S
price: 1025

item_no: 100185305
location: 105
location_type: S
price: 19266.69

item_no: 100035000
location: 607
location_type: S
price: 275.25

item_no: 100035001
location: 607
location_type: S
price: 275.25

item_no: 100040000
location: 607
location_type: S
price: 275.25


user:  item info
bot:  which of the details of the below you want :
user:  /price_details
bot:  Select Get item prices, to retrieve prices for all items at all locations.

Select Get s

In [8]:
intents=set()
intentdictnew=dict()
for conv in conversations:
    for i in conv:
        if i['event']=="user":
            if i['parse_data']['intent']['name'] not in intents:
                intents.add(i['parse_data']['intent']['name'])
                intentdictnew[i['parse_data']['intent']['name']]={i["text"]}
            else:
                intentdictnew[i['parse_data']['intent']['name']].add(i["text"])
for k,v in intentdictnew.items():
    print(k,":",v)
#NEED TO REMOVE INTENTS CREATED BY BUTTONS



greet : {'hi', "how's the weather", 'hey', 'sup', "how's the weather like"}
user_query : {'/item_details', 'item info]', 'item info'}
price_details : {'price detaosl', 'price detaisl', 'price details', '/price_details'}
get_item_prices : {'item ifno', '/get_item_prices'}
get_item_all_locs : {'cost of all items in particular location', '/get_item_all_locs'}
nlu_fallback : {'UPC', '100150000', 'president of india', 'prime minister of tamilnadu', 'prime minister of india', 'cheif minister of tamilnadu'}
hello : {'/hello'}
get_specific_price : {'/get_specific_price', 'particular item particular location', 'cost of particular item'}
item_details : {'100150000', '100185305', '100035001', '607', '105', '1000350001'}
location_details : {'location 105', 'location 607'}
others : {'what is the weather like', 'what is the weather in trichy', 'what is the weather', 'what is the weather in chennai', '/others'}
location_info : {'100035001', '100150000', 'location 246001'}
item_info : {'100150000', '1

https://www.kaggle.com/datasets/salmanfaroz/small-talk-intent-classification-data

reading the nlu yaml file

In [45]:
import yaml

with open(r'D:\documents\internship\rasachatbot\data\nlu.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    NLU = yaml.full_load(file)
print(NLU)
nlu=NLU["nlu"]
print(nlu)

{'version': '3.1', 'nlu': [{'intent': 'greet', 'examples': '- hey\n- hello\n- hi\n- hello there\n- good morning\n- good evening\n- moin\n- hey there\n- hey dude\n- good morning\n- good evening\n- good afternoon\n- sup\n'}, {'intent': 'goodbye', 'examples': '- cu\n- good by\n- cee you later\n- good night\n- bye\n- goodbye\n- have a nice day\n- see you around\n- bye bye\n- see you later\n'}, {'intent': 'affirm', 'examples': '- yes\n- y\n- indeed\n- of course\n- that sounds good\n- correct\n'}, {'intent': 'deny', 'examples': "- no\n- n\n- never\n- I don't think so\n- don't like that\n- no way\n- not really\n"}, {'intent': 'mood_great', 'examples': '- perfect\n- great\n- amazing\n- feeling like a king\n- wonderful\n- I am happy\n- I am feeling very good\n- I am great\n- I am amazing\n- I am going to save the world\n- super stoked\n- extremely good\n- so so perfect\n- so good\n- so perfect\n'}, {'intent': 'mood_unhappy', 'examples': "- my day was horrible\n- I am sad\n- I don't feel very we

In [46]:
intentdict=dict()
for i in nlu:
    temp=i["examples"].split("\n- ")
    temp[0]=temp[0][2:]
    temp[-1]=temp[-1][:-1]
    intentdict[i["intent"]]=set(temp)
for k,v in intentdict.items():
    print(k,": ",v)
#print(intentdict)

greet :  {'hey dude', 'moin', 'hi', 'good morning', 'hello', 'hey', 'good evening', 'sup', 'hello there', 'good afternoon', 'hey there'}
goodbye :  {'cee you later', 'have a nice day', 'bye bye', 'cu', 'goodbye', 'good by', 'bye', 'see you later', 'good night', 'see you around'}
affirm :  {'indeed', 'correct', 'of course', 'yes', 'y', 'that sounds good'}
deny :  {'n', 'not really', 'no', "don't like that", "I don't think so", 'never', 'no way'}
mood_great :  {'I am great', 'so so perfect', 'so perfect', 'amazing', 'super stoked', 'I am amazing', 'extremely good', 'perfect', 'so good', 'wonderful', 'I am happy', 'I am going to save the world', 'great', 'I am feeling very good', 'feeling like a king'}
mood_unhappy :  {'not good', 'not very good', 'I am sad', 'super sad', 'so saad', "I'm so sad", 'I am disappointed', 'so sad', 'extremly sad', "I don't feel very well", 'sad', 'my day was horrible', 'unhappy', 'very sad'}
bot_challenge :  {'am I talking to a human?', 'are you a bot?', 'are 

add the new user inputs to the respective intents

In [47]:
knew=set(intentdictnew.keys())
k=set(intentdict.keys())
inter=k.intersection(knew)
for i in inter:
    intentdict[i].update(intentdictnew[i])
for k,v in intentdict.items():
    print(k,": ",v)


greet :  {'hey dude', 'moin', 'hi', 'good morning', "how's the weather", 'hello', 'hey', 'good evening', 'sup', 'hello there', "how's the weather like", 'good afternoon', 'hey there'}
goodbye :  {'cee you later', 'have a nice day', 'bye bye', 'cu', 'goodbye', 'good by', 'bye', 'see you later', 'good night', 'see you around'}
affirm :  {'indeed', 'correct', 'of course', 'yes', 'y', 'that sounds good'}
deny :  {'n', 'not really', 'no', "don't like that", "I don't think so", 'never', 'no way'}
mood_great :  {'I am great', 'so so perfect', 'so perfect', 'amazing', 'super stoked', 'I am amazing', 'extremely good', 'perfect', 'so good', 'wonderful', 'I am happy', 'I am going to save the world', 'great', 'I am feeling very good', 'feeling like a king'}
mood_unhappy :  {'not good', 'not very good', 'I am sad', 'super sad', 'so saad', "I'm so sad", 'I am disappointed', 'so sad', 'am sad', 'extremly sad', "I don't feel very well", 'sad', 'my day was horrible', 'unhappy', 'very sad', 'im happy'}


updating the NLU dictionary

In [48]:
temp=[]
for k,v in intentdict.items():
    s="- "+"\n- ".join(v)+"\n"
    d=dict()
    d["intent"]=k
    d["examples"]=s
    temp.append(d)
print(temp)
print("old:")
print(NLU)
NLU['nlu']=temp
print("updated:")
print(NLU)


[{'intent': 'greet', 'examples': "- hey dude\n- moin\n- hi\n- good morning\n- how's the weather\n- hello\n- hey\n- good evening\n- sup\n- hello there\n- how's the weather like\n- good afternoon\n- hey there\n"}, {'intent': 'goodbye', 'examples': '- cee you later\n- have a nice day\n- bye bye\n- cu\n- goodbye\n- good by\n- bye\n- see you later\n- good night\n- see you around\n'}, {'intent': 'affirm', 'examples': '- indeed\n- correct\n- of course\n- yes\n- y\n- that sounds good\n'}, {'intent': 'deny', 'examples': "- n\n- not really\n- no\n- don't like that\n- I don't think so\n- never\n- no way\n"}, {'intent': 'mood_great', 'examples': '- I am great\n- so so perfect\n- so perfect\n- amazing\n- super stoked\n- I am amazing\n- extremely good\n- perfect\n- so good\n- wonderful\n- I am happy\n- I am going to save the world\n- great\n- I am feeling very good\n- feeling like a king\n'}, {'intent': 'mood_unhappy', 'examples': "- not good\n- not very good\n- I am sad\n- super sad\n- so saad\n- I

In [49]:
with open(r'D:\documents\internship\rasachatbot\data\nlu2.yml', 'w') as file:
    documents = yaml.dump(NLU, file)